<a href="https://colab.research.google.com/github/jsoldate44/stable-diffusion/blob/main/Becca's_CueGenerator_Drivedown_6_13_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh

In [ ]:
!chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh

!bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local

# add imagemagick
!apt install imagemagick

In [ ]:
!conda init bash

In [ ]:
# Clone the repo
import os
from IPython.display import Image
from google.colab import files as FILE
from google.colab import drive

os.chdir('/content')
drive.mount('/content/drive')
!git clone --branch neg_prompt https://github.com/jsoldate44/stable-diffusion.git
os.chdir('stable-diffusion')

In [ ]:
!conda env update -n base -f environment.yaml

In [ ]:
!curl https://f004.backblazeb2.com/file/aai-blog-files/sd-v1-4.ckpt > sd-v1-4.ckpt

# Generating Images with Stable Diffusion

Setup for the list of prompts and the directory of resulting images

In [ ]:
# either set all images here
# generating cell
Prompt_list = [
    {'fold':'Cue00_','seed':5545,'prompt':'Car windshield with a bus outside'},
    {'fold':'Cue01_','seed':5546,'prompt':'inside of car with school bus stop outside'},
    {'fold':'Cue02_','seed':5547,'prompt':'inside of car with neighborhood outside'},
    {'fold':'Cue03_','seed':5548,'prompt':'school bus approaching in the distance down a quiet street'}
    ]

remove_prompt = '((((ugly)))), (outside), (((duplicate))), ((morbid)), ((mutilated)), out of frame, extra fingers, mutated hands, ((poorly drawn hands)), ((poorly drawn face)), (((mutation))), (((deformed))), ((ugly)), blurry, ((bad anatomy)), (((bad proportions))), ((extra limbs)), cloned face, (((disfigured))), extra limbs, (bad anatomy), gross proportions, (malformed limbs), ((missing arms)), ((missing legs)), (((extra arms))), (((extra legs))), mutated hands, (fused fingers), (too many fingers), (((long neck)))'

In [ ]:
# Or add individual prompts here
Prompt_list.append(
    {'fold':'SomethingElse','seed':5323,'prompt':'Sitting on the beach'}
    )

for i in Prompt_list:
  print(i['prompt'])

In [ ]:
# use this to clear the list of prompt when you want to start over
# only run this as necessary
Prompt_list = []

In [ ]:
# Get rid of the negative prompt
remove_prompt = ''

Actually generating the images

In [ ]:
for param in Prompt_list:
  # separate different prompts
  fname = param['fold']
  newseed = param['seed']
  prompt = param['prompt']

  !python "scripts/txt2img.py" \
  --plms \
  --ckpt sd-v1-4.ckpt \
  --skip_grid \
  --n_samples 1  \
  --n_iter 50 \
  --outdir $fname$newseed \
  --seed $newseed \
  --ddim_steps 50 \
  --prompt "$prompt" \
  --unprompt "$remove_prompt"

  # name the images
  imgs = os.listdir(f'{fname}{newseed}/samples')
  montdir = f'{fname}{newseed}/montsamples'
  workdir = f'{fname}{newseed}/samples'

  try:
    os.mkdir(montdir)
  except:
    pass

  for i in imgs:
    the_image = f'{workdir}/{i}'
    new_image = f'{montdir}/{i}'
    lab = i.split('.')[0]
    !convert $the_image -gravity south -pointsize 50 -stroke '#000C' -strokewidth 4 -annotate 0 "$lab" -pointsize 50 -stroke none -fill white -annotate 0 "$lab" $new_image

  !montage -tile 5x10 -geometry 200x200+5+5 $(find /content/stable-diffusion/$fname$newseed/montsamples/ -type f | sort) /content/stable-diffusion/$fname$newseed/montage.png
  !rm -rf $montdir

  !zip -r /content/stable-diffusion/{fname}{newseed}.zip /content/stable-diffusion/{fname}{newseed}
  #FILE.download(f'/content/stable-diffusion/{fname}{newseed}.zip')

  ## the directory /My Drive/VirginiaTech/RP2B/CoLab_Downloads must exist for this not to fail
  # be certain
  !cp /content/stable-diffusion/{fname}{newseed}.zip /content/drive/My\ Drive/VirginiaTech/RP2B/CoLab_Downloads

If you generated multiple images (or ran the above cell multiple times with different promtps) you can view different images by changing the number `00000` in the command below. E.g. `00001.png` will display the *second* generated image.

Useful bits of code for checking the output

In [ ]:
# Use this to move through the prompts you specify
check_prompt = 0

# Should specify which folder you are examining
out_fold = Prompt_list[check_prompt]['fold']
out_seed = Prompt_list[check_prompt]['seed']
print(f'Display folder is: {out_fold}{out_seed}')

for i in Prompt_list:
  print(f"Folder: {i['fold']}, Prompt: {i['prompt']}")

In [ ]:
     # display the full montage
Image(f'/content/stable-diffusion/{out_fold}{out_seed}/montage.png')

In [ ]:
# Use this to move through individual images, starting at 00000, you only need to give a number
img_number = 27

# print the image based on check_prompt and img_number
Image(f'/content/stable-diffusion/{out_fold}{out_seed}/samples/{img_number:05}.png')

# Downloading Images

Downloading the images you just viewed

Relies on cells [Promp](https://colab.research.google.com/drive/15x3RufhDhjYbcg7SPIKDI9z908kB91R3#scrollTo=y5HRAoN7blGY&line=4&uniqifier=1)
and [Image](https://colab.research.google.com/drive/15x3RufhDhjYbcg7SPIKDI9z908kB91R3#scrollTo=vrD4jmM7XesE&line=1&uniqifier=1)

rerun them to change the download location



In [ ]:
# download the image you just viewed
from google.colab import files as FILE
FILE.download(f"/content/stable-diffusion/{out_fold}{out_seed}/samples/{img_number:05}.png")

Download a zip of the whole thing

In [ ]:
!zip -r /content/stable-diffusion/{out_fold}{out_seed}.zip /content/stable-diffusion/{out_fold}{out_seed}
FILE.download(f'/content/stable-diffusion/{out_fold}{out_seed}.zip')

You can delete the contents of the `samples` folder and `samples.zip` to keep the directory clean if you want to run multiple samplings with different prompts.

The below commands are commented out by default to avoid accidental deletion - uncomment them and run the cell in order to delete all of the generated images / ZIP file.

In [ ]:
for prompts in Prompt_list:
  !rm -rf /content/stable-diffusion/prompts[out_fold]prompts[out_seed]
  !rm /content/stable-diffusion/*.zip

To run another prompt, simply start at the **Generating Images with Stable Diffusion** section and follow along as before!

# Final Words

That's all it takes to generate images using the new Stable Diffusion model - don't forget to share your fun creations with us on [Twitter](https://twitter.com/AssemblyAI)! If you want to learn more about how Stable Diffusion works, you can check out our [Introduction to Diffusion Models for Machine Learning](https://www.assemblyai.com/blog/diffusion-models-for-machine-learning-introduction/) article. If you enjoyed this article, feel free to check out more of our [blog](https://www.assemblyai.com/blog/) or [YouTube channel](https://www.youtube.com/c/AssemblyAI) for Machine Learning content, or feel free to [follow our newsletter](https://assemblyai.us17.list-manage.com/subscribe?u=cb9db7b18b274c2d402a56c5f&id=2116bf7c68) to stay in the loop for new releases.

In [ ]:
print(remove_prompt)